In [1]:
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table, no_update, ALL, MATCH, callback_context
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

In [36]:
predictions = pd.read_csv('predictions.csv', index_col = 'player')


mean_of_means = pd.read_csv('mean_of_means.csv', index_col = 0).iloc[:,0]
var_of_means = pd.read_csv('var_of_means.csv', index_col = 0).iloc[:,0]
mean_of_vars = pd.read_csv('mean_of_vars.csv', index_col = 0).iloc[:,0]

#three tabs:

- Player info, defaults to prediction. Scrolling list of all eligible players, with editable prediction stats. Converts to Z-scores, G-scores, and X-scores, as well as Z-effective, G-effective, and X-effective
- Drafting. 13 x 12, with three boxes at the top for recommendations by G,Z, and H. Every time one of the 13 x 12 boxes is changed, the algorithms get run and the top players are selected 
- Auction value. 

In [24]:

app = JupyterDash(__name__)

drafters = list(range(1,13))
picks = list(range(1,14))

width = str(100/len(drafters)) + '%'

#add a tab for player info 
#for now just use the g-scores


titles_by_drafter = [html.H1([html.H1(str(i))]
                            ,style={'textAlign': 'center','fontSize' : 10,'width' : width,'display': 'inline-block'}
                            )
                    for i in drafters
                   ]


#change the ID structure from (type, index) to (type, drafter, pick)
dropdowns_by_drafter = [ 
                        html.Div(
                            [html.Div(id = {'type' : 'dropdown-parent'
                                             ,'drafter' : i
                                             ,'pick' : j},
                                children = dcc.Dropdown(id = {'type' : 'dropdown-selection'
                                                              ,'drafter' : i
                                                              ,'pick' : j}
                                                 ) 
                                                    ,style = {'justify_content' : 'center'
                                                              ,'align-items' : 'center'
                                                              ,'width' : '90%'
                                                              ,'padding-left' : '5%'
                                                              ,'padding-right' : '5%'
                                                              ,'height' : '5vh'}
                                          )
                                 for j in picks
                            ] 
                                ,style={'textAlign': 'center'
                                        ,'fontSize' : 10
                                        ,'width' : width
                                        ,'display': 'inline-block'}
                               )
                            for i in drafters
                       ]

app.layout = html.Div([
    html.H1(children='Draft Simulator', style={'textAlign':'center'})
    ,html.H1(titles_by_drafter)
    ,html.H1(dropdowns_by_drafter)
    ,dcc.Store(id='player-data', data = df.to_dict('index'))
    ,dcc.Store(id='players-available', data = df.index.values)
    ,dcc.Store(id='current-pick', data = {'player' : 1, 'pick' : 1 })

])

@callback(
    Output('players-available', 'data'),
    State('player-data', 'data'),
    Input({"type": "dropdown-selection", "drafter": ALL , "pick" : ALL}, "value"),
    prevent_initial_call=True
)
def update_available_players(player_data,selected_players):
    
    all_players = pd.Series(player_data.keys())
    res = all_players[~all_players.isin(selected_players)]
    return res


@callback(
    Output({"type": "dropdown-selection", "drafter": MATCH, "pick" : MATCH}, 'options'),
    State('players-available', 'data'),
    State({"type": "dropdown-selection", "drafter": MATCH, "pick" : MATCH}, 'value'),
    Input({"type": "dropdown-parent", "drafter": MATCH, "pick" : MATCH}, "n_clicks"),
    prevent_initial_call=True
)
def update_dropdown(available_players,dropdown_value, _):
    
    if dropdown_value is None:
        all_players = available_players
    else:
        all_players = [dropdown_value] + available_players 
        
    return[{'label' : player, 'value' : player} for player in all_players] 

@callback(
    Output('current-pick', 'data'),
    Input({"type": "dropdown-selection", "drafter": ALL, "pick" : ALL}, 'value'),
    prevent_initial_call=True
)
def update_current_pick(selected_players):
    
    trigger = callback_context.triggered_id
    drafter = trigger['drafter']
    pick = trigger['pick']
    
    if pick%2 ==1:
        if drafter ==12:
            new_drafter = 12
            new_pick = pick + 1
        else:
            new_drafter = drafter + 1
            new_pick = pick
    else:
        if drafter == 1:
            new_drafter = 1
            new_pick = pick + 1
        else:
            new_drafter = drafter - 1
            new_pick = pick

    return {'drafter' : new_drafter, 'pick' : new_pick}
   
@callback(
    Output({"type": "dropdown-parent", "drafter": MATCH, "pick" : MATCH}, 'style'),
    Input('current-pick', 'data'),
    State({"type": "dropdown-selection", "drafter": MATCH, "pick" : MATCH}, 'value'),

    prevent_initial_call=True
)
def show_current_pick(current_pick, _):
    
    current_drafter = current_pick['drafter']
    current_pick = current_pick['pick']
    
    output_id = callback_context.outputs_list[0]['id']
    
    if (output_id['drafter'] == current_drafter) & (output_id['pick'] == current_pick):
        color = 'blue'
    else:
        color = None
    
    return {'justify_content' : 'center'
              ,'align-items' : 'center'
              ,'width' : '90%'
              ,'padding-left' : '5%'
              ,'padding-right' : '5%'
              ,'height' : '5vh'
              ,'color' : color}

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


In [8]:
df = pd.read_csv('predictions.csv').sort_values('g_total', ascending = False).set_index('player')

app = JupyterDash(__name__)

drafters = list(range(1,13))
picks = list(range(1,14))

width = str(100/len(drafters)) + '%'


titles_by_drafter = [html.H1([html.H1(str(i))]
                            ,style={'textAlign': 'center','fontSize' : 10,'width' : width,'display': 'inline-block'}
                            )
                    for i in drafters
                   ]


#change the ID structure from (type, index) to (type, drafter, pick)
dropdowns_by_drafter = [ 
                        html.Div(
                            [html.Div(id = {'type' : 'dropdown-parent'
                                             ,'drafter' : i
                                             ,'pick' : j},
                                children = dcc.Dropdown(id = {'type' : 'dropdown-selection'
                                                              ,'drafter' : i
                                                              ,'pick' : j}
                                                 ) 
                                                    ,style = {'justify_content' : 'center'
                                                              ,'align-items' : 'center'
                                                              ,'width' : '90%'
                                                              ,'padding-left' : '5%'
                                                              ,'padding-right' : '5%'
                                                              ,'height' : '5vh'}
                                          )
                                 for j in picks
                            ] 
                                ,style={'textAlign': 'center'
                                        ,'fontSize' : 10
                                        ,'width' : width
                                        ,'display': 'inline-block'}
                               )
                            for i in drafters
                       ]

app.layout = html.Div([
    html.H1(children='Draft Simulator', style={'textAlign':'center'})
    ,html.H1(titles_by_drafter)
    ,html.H1(dropdowns_by_drafter)
    ,dcc.Store(id='player-data', data = df.to_dict('index'))
    ,dcc.Store(id='players-available', data = df.index.values)

])

@callback(
    Output('players-available', 'data'),
    State('player-data', 'data'),
    Input({"type": "dropdown-selection", "drafter": ALL , "pick" : ALL}, "value"),
    prevent_initial_call=True
)
def update_available_players(player_data,selected_players):
    
    all_players = pd.Series(player_data.keys())
    res = all_players[~all_players.isin(selected_players)]
    return res


@callback(
    Output({"type": "dropdown-selection", "drafter": MATCH, "pick" : MATCH}, 'options'),
    State('players-available', 'data'),
    State({"type": "dropdown-selection", "drafter": MATCH, "pick" : MATCH}, 'value'),
    Input({"type": "dropdown-parent", "drafter": MATCH, "pick" : MATCH}, "n_clicks"),
    prevent_initial_call=True
)
def update_dropdown(available_players,dropdown_value, _):
    
    if dropdown_value is None:
        all_players = available_players
    else:
        all_players = [dropdown_value] + available_players 
        
    return[{'label' : player, 'value' : player} for player in all_players] 

if __name__ == '__main__':
    app.run_server(debug=True)

0

In [4]:
! pip install flask==2.2.2

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Zach Rosenof\\Anaconda3\\Lib\\site-packages\\~arkupsafe\\_speedups.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [3]:
! pip install Jinja2==3.0.3

  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


In [5]:
! pip install dash-core-components